# 3.2 Solving for a double pendulum
owo

In [11]:
import numpy as np
import matplotlib.pyplot as plt

### Unterpunkt Zero: RK4 herkopieren
weil Importieren aus notebooks is a krampf

In [12]:
#Funktion für das explizite Runge-Kutta-Verfahren 
def rk_explicit(F, y0, t, e, A, b, c):
    """
    exp. Runge–Kutta für y' =F(y, t).
    y0: 1D array
    t:  1D array der Zeit mt Schritten e (epsilon xD)
    A,b,c: Butcher coeff
    """
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    c = np.array(c, dtype=float)

    s = len(b) 
    y0 = np.array(y0, dtype=float)
    dim = len(y0)

    Y = np.zeros((len(t), dim))
    Y[0] = y0

    for n in range(len(t) - 1):
        yn = Y[n]
        tn = t[n]

        K = np.zeros((s, dim))  # stage derivatives

        for i in range(s):
            y_stage = yn.copy()
            for j in range(i):  # explizit nur j < i
                y_stage += e * A[i, j] * K[j]

            K[i] = F(y_stage, tn + c[i] * e)

        Y[n + 1] = yn.copy()
        for i in range(s):
            Y[n + 1] += e * b[i] * K[i]

    return Y

#Butcher Tableus
#zerst Euler
A_eu = [[0.0]]
b_eu = [1.0]
c_eu = [0.0]

#Runge Kutta Odg 4
A_rk4 = [
    [0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.0, 0.0],
    [0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 1.0, 0.0],
]
b_rk4 = [1/6, 1/3, 1/3, 1/6]
c_rk4 = [0.0, 0.5, 0.5, 1.0]

### Unterpunkt a: Implement the Functions and solve with RK4

In [13]:
w_sq = 9.81  # omega^2 = g/l, und l=1

def F_duop(angabe, t):
    th1, th2, p1, p2 = angabe
    th = th1 - th2
    sin, cos = np.sin(th), np.cos(th)

    th1_dot= (p1 - p2*cos) /(1 + sin**2)
    th2_dot = (2*p2 - p1*cos) / (1 + sin**2)

    A = (p1*p2*sin) /(1 + sin**2)
    B =((p1**2 + 2*p2**2 - 2*p1*p2*cos) / ((1 + sin**2)**2)) * sin*cos

    p1_dot =-A + B - 2*w_sq*np.sin(th1)
    p2_dot =A - B -1*w_sq*np.sin(th2)

    return np.array([th1_dot, th2_dot, p1_dot, p2_dot], dtype=float)

# Zeit udn Schrittweite
e = 0.001
t = np.arange(0, 50+e, e)
#Anfangswerte aus angabe
y0_1 =[0.0, 0.0, 4.0, 2.0]
y0_2= [0.0, 0.0, 0.0, 4.0]

Y_1 = rk_explicit(F_duop, y0_1, t, e, A_rk4, b_rk4, c_rk4)
Y_2 = rk_explicit(F_duop, y0_2, t, e, A_rk4, b_rk4, c_rk4)
